In [5]:
# Cobain dari youtube
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [4]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [6]:
driver = webdriver.Chrome()

# Open the Tokopedia URL
url = 'https://www.tokopedia.com/search?st=&q=nugget%20ikan&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&navsource='
driver.get(url)

# Preprocessing data and declaring variable
data = []
badge_mapping = {
    "https://images.tokopedia.net/img/official_store_badge.png": "official store"
    , "https://images.tokopedia.net/img/goldmerchant/pm_activation/badge/PM%20Pro%20Small.png": "gold merchant"
    , "https://images.tokopedia.net/img/power_merchant_badge.png": "power merchant"
}

for i in range(30) : 
    WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#zeus-root")))
    time.sleep(2)
    
    for j in range (21): 
        driver.execute_script("window.scrollBy(0, 250)")
        time.sleep(1)

    driver.execute_script("window.scrollBy(50, 0)")
    time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    products = soup.find_all('div', class_="css-5wh65g")

    # Extract product information
    for product in products:
        # Extract product name
        product_name_tag = product.find('span', class_='_0T8-iGxMpV6NEsYEhwkqEg==')
        product_name = product_name_tag.text.strip() if product_name_tag else 'N/A'

        # Extract price
        price_tag = product.find('div', class_='_67d6E1xDKIzw+i2D2L0tjw==')
        price = price_tag.text.strip() if price_tag else 'N/A'

        # Extract Location
        location_tag = product.find('span', class_='pC8DMVkBZGW7-egObcWMFQ== flip')
        location = location_tag.text.strip() if location_tag else 'N/A'        
        
        # Extract shop name
        shop_name_tag = product.find('span', class_='T0rpy-LEwYNQifsgB-3SQw== pC8DMVkBZGW7-egObcWMFQ== flip')
        shop_name = shop_name_tag.text.strip() if shop_name_tag else 'N/A'

        # Extract shop badge
        shop_badge_tag = product.find('img', class_='YtXczlnkXDXQ59u3vhDxiA==')
        shop_badge_src = shop_badge_tag['src'] if shop_badge_tag else None
        shop_badge = badge_mapping.get(shop_badge_src, "N/A")

        # Extract rating
        rating_tag = product.find('span', class_='_9jWGz3C-GX7Myq-32zWG9w==')
        rating = rating_tag.text.strip() if rating_tag else 'N/A'

        # Extract number of items sold
        sold_tag = product.find('span', class_='se8WAnkjbVXZNA8mT+Veuw==')
        sold = sold_tag.text.strip() if sold_tag else 'N/A'

        # Store the data in a set to avoid duplicates
        data.append(
            (product_name, price, location, shop_name, shop_badge, rating, sold)
        )
    
    time.sleep(2)

    # Check if "Laman Berikutnya" disabled 
    next_button = driver.find_element(By.CSS_SELECTOR, "button[aria-label='Laman berikutnya']")
    if next_button.get_attribute("disabled") : 
        print("Last page reached. Exiting Loop")
        break

    driver.find_element(By.CSS_SELECTOR, "button[aria-label='Laman berikutnya']").click()
    time.sleep(4)


df = pd.DataFrame(data, columns = ['Product_name', 'price', 'location', 'shop_name', 'shop_badge', 'rating', 'sold'])

driver.quit()

Last page reached. Exiting Loop


In [7]:
df = pd.DataFrame(data, columns = ['Product_name', 'price', 'location', 'shop_name', 'shop_badge', 'rating', 'sold'])

In [8]:
df.to_excel('tokopedia_all_products.xlsx')